Article from Philippe:
https://practicalcheminformatics.blogspot.com/2023/06/getting-real-with-molecular-property.html


In [2]:
# for running in colab:

# !pip install rdkit
# !pip install deepchem


# !wget https://raw.githubusercontent.com/NinaB99/AI-for-Chemistry/main/Data/ADME_public_set_3521.csv
# !wget https://raw.githubusercontent.com/NinaB99/AI-for-Chemistry/main/Data/11095_2013_1222_MOESM2_ESM.csv

# load biogen data
# import pandas as pd

# biogen_data=pd.read_csv("ADME_public_set_3521.csv")
# print(biogen_data.columns)

# #load bioavailabity data
# bio_avail_data = pd.read_csv("11095_2013_1222_MOESM2_ESM.csv",sep=";")
# print(bio_avail_data.columns)

In [19]:
# load biogen data
import pandas as pd

biogen_data=pd.read_csv("../Data/Biogen.csv")
print(biogen_data.columns)

#new data
data = pd.read_csv("../Data/CuratedSol.csv")
print(data.columns)

#load bioavailabity data
bio_avail_data = pd.read_csv("../Data/Bioavailibility.csv")
#print(bio_avail_data.columns)



Index(['Internal ID', 'Vendor ID', 'SMILES', 'CollectionName',
       'LOG HLM_CLint (mL/min/kg)', 'LOG MDR1-MDCK ER (B-A/A-B)',
       'LOG SOLUBILITY PH 6.8 (ug/mL)',
       'LOG PLASMA PROTEIN BINDING (HUMAN) (% unbound)',
       'LOG PLASMA PROTEIN BINDING (RAT) (% unbound)',
       'LOG RLM_CLint (mL/min/kg)', 'MolW(Da)', 'NumHAcceptors', 'NumHDonors',
       'LogP', 'Lipinski_rule'],
      dtype='object')
Index(['ID', 'Name', 'InChI', 'InChIKey', 'SMILES', 'Solubility', 'SD',
       'Ocurrences', 'Group', 'MolWt', 'MolLogP', 'MolMR', 'HeavyAtomCount',
       'NumHAcceptors', 'NumHDonors', 'NumHeteroatoms', 'NumRotatableBonds',
       'NumValenceElectrons', 'NumAromaticRings', 'NumSaturatedRings',
       'NumAliphaticRings', 'RingCount', 'TPSA', 'LabuteASA', 'BalabanJ',
       'BertzCT', 'MolW(Da)', 'LogP', 'Lipinski_rule'],
      dtype='object')


use feed-forward NN using pytorch.
train on solubility from dataset 1 first.
then optimize for bioavailability

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
from deepchem.feat import RDKitDescriptors
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



In [3]:
def canonicalize(Dataframe: pd.DataFrame):
    
    """Canonicalizes the SMILES from Dataframe. A column called 'SMILES' is requiered

    Args: Dataframe with 'SMILES' column contaning smiles. 
    """
    
    Dataframe['SMILES'] = Dataframe['SMILES'].apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x))) #canonicalize smiles from a Dataframe                                          
    

In [4]:

# they are both log so I do not need to log-transform
canonicalize(biogen_data)
canonicalize(curated_data)

#make sure to remove overlaps
import pandas as pd
import numpy as np

# Sample DataFrames (replace with your actual data)
# biogen_data = pd.read_csv('biogen_data.csv')
# curated_data = pd.read_csv('curated_data.csv')

# Rename columns for consistency
biogen_data_new = biogen_data.rename(columns={"LOG SOLUBILITY PH 6.8 (ug/mL)": "Log solubility"})
curated_data_new = curated_data.rename(columns={"Solubility": "Log solubility"})

# Select relevant columns
biogen_data_new = biogen_data_new[["SMILES", "Log solubility"]]
curated_data_new = curated_data[["SMILES", "Log solubility"]]

# Combine the DataFrames, prioritizing the curated_data
data = pd.concat([curated_data_new, biogen_data_new])

# Drop duplicate SMILES, keeping the first occurrence (curated_data)
data = data.drop_duplicates(subset="SMILES", keep='first')

# Reset index for the final combined DataFrame
#combined_data = combined_data.reset_index(drop=True)

# Print the combined DataFrame
print(data)



[13:06:09] WARNING: not removing hydrogen atom without neighbors
[13:06:09] WARNING: not removing hydrogen atom without neighbors
[13:06:09] WARNING: not removing hydrogen atom without neighbors
[13:06:09] WARNING: not removing hydrogen atom without neighbors
[13:06:09] WARNING: not removing hydrogen atom without neighbors
[13:06:09] WARNING: not removing hydrogen atom without neighbors
[13:06:09] WARNING: not removing hydrogen atom without neighbors
[13:06:09] WARNING: not removing hydrogen atom without neighbors
[13:06:09] WARNING: not removing hydrogen atom without neighbors
[13:06:09] WARNING: not removing hydrogen atom without neighbors
[13:06:09] WARNING: not removing hydrogen atom without neighbors
[13:06:09] WARNING: not removing hydrogen atom without neighbors
[13:06:09] WARNING: not removing hydrogen atom without neighbors
[13:06:09] WARNING: not removing hydrogen atom without neighbors
[13:06:09] WARNING: not removing hydrogen atom without neighbors
[13:06:09] WARNING: not r

                                                 SMILES  Log solubility  \
206                                                 CCO        0.209992   
229                                   CNC(=S)[S-].[Na+]       -0.291255   
262                      CCCC[N+](CCCC)(CCCC)CCCC.[Br-]        0.286700   
267                          Nc1cnn(CCO)c1N.O=S(=O)(O)O       -0.811347   
268    O=S(=O)([O-])c1cccc(S(=O)(=O)[O-])c1.[Na+].[Na+]       -0.991695   
...                                                 ...             ...   
3516            O=C(c1ccc2c(c1)CCCC2)N1CCOCC1c1ccn[nH]1             NaN   
3517           O=C(Nc1nc2ccccc2[nH]1)c1ccc(-n2cccc2)cc1             NaN   
3518         NC(=O)c1noc([C@@H](CCCC2CCCCC2)CC(=O)NO)n1             NaN   
3519         CCCCCCCCc1ccc(CC[C@](N)(CO)COP(=O)(O)O)cc1             NaN   
3520  Cc1cccc(/C=N/Nc2cc(N3CCOCC3)n3nc(-c4ccncc4)cc3...             NaN   

     Internal ID  Vendor ID CollectionName  LOG HLM_CLint (mL/min/kg)  \
206          NaN        Na

In [ ]:
len(data["SMILES"])

In [20]:

#### DATA PREPARATION ####
# try with new data

# Function to generate features from SMILES strings using RDKit descriptors
def generate_features(smiles_list):
    featurizer = RDKitDescriptors()
    features = featurizer.featurize(smiles_list)
    # Drop features containing invalid values
    features = features[:, ~np.isnan(features).any(axis=0)]
    return features

#remove nan values from data
data = data.dropna(subset=['Solubility'])

#get x and y data (x is the molecular descriptors, y is the solubility)
y_data = data["Solubility"]

print(len(y_data))

# Generate features from SMILES data (get smiles from df)
smiles = data["SMILES"]
X_data = generate_features(smiles)

#split data into training and validation using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)
# Convert y pandas Series to NumPy array
y_train = y_train.values.astype(np.float32)
y_test = y_test.values.astype(np.float32)

#scale x values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#convert data to pytorch tensors (like numpy arrays but for pytorch)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Reshape the target tensor to match the shape of the output tensor
y_train_tensor = y_train_tensor.view(-1, 1)
# Reshape the target tensor to match the shape of the output tensor
y_test_tensor = y_test_tensor.view(-1, 1)


9982


[14:28:49] WARNING: not removing hydrogen atom without neighbors
[14:28:49] WARNING: not removing hydrogen atom without neighbors
[14:28:49] WARNING: not removing hydrogen atom without neighbors
[14:28:51] WARNING: not removing hydrogen atom without neighbors
[14:28:51] WARNING: not removing hydrogen atom without neighbors
[14:28:51] WARNING: not removing hydrogen atom without neighbors
[14:28:51] WARNING: not removing hydrogen atom without neighbors
[14:28:51] WARNING: not removing hydrogen atom without neighbors
[14:28:51] WARNING: not removing hydrogen atom without neighbors
[14:28:53] WARNING: not removing hydrogen atom without neighbors
[14:28:53] WARNING: not removing hydrogen atom without neighbors
[14:28:53] WARNING: not removing hydrogen atom without neighbors
[14:28:53] WARNING: not removing hydrogen atom without neighbors
[14:28:53] WARNING: not removing hydrogen atom without neighbors
[14:28:53] WARNING: not removing hydrogen atom without neighbors
[14:28:53] WARNING: not r

In [21]:
X_data.shape[1]

198

In [14]:
np.std(y_data)

2.368035824195333

In [26]:
#### CREATING MODEL ####

#haven't tried it with the newest settings

# Define the neural network architecture
class SolubilityPredictor(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SolubilityPredictor, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Define hyperparameters
input_dim = X_data.shape[1]  # Number of molecular descriptors. 201 different ones
hidden_dim = 256   #best according to grid search
output_dim = 1
learning_rate = 0.001 #gets overruled by the grid search
num_epochs = 800
batch_size = 32

#create dataloader for batch training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#initialize model, loss function (nn.MSELoss) and optimizer
model = SolubilityPredictor(input_dim, hidden_dim, output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

##### Grid-search for best hyperparameters #####
# Define your hyperparameter grid
param_grid = {
    'hidden_layer_sizes': [(256,256,256),(512,512,512)],  # Number of neurons in the hidden layer(s)
    'activation': ['relu', 'tanh'],  # Activation function
    'solver': ['adam', 'sgd'],  # Optimization algorithm
    'learning_rate': ['constant', 'adaptive'],  # Learning rate schedule
}

# Create an MLPRegressor object
mlp = MLPRegressor(max_iter=num_epochs, batch_size=batch_size)

# Perform grid search
grid_search = GridSearchCV(mlp, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters found
print("Best Hyperparameters:", grid_search.best_params_)

# Get the best model
model = grid_search.best_estimator_

# Evaluate the best model on the test set
test_score = model.score(X_test, y_test)
print("Test Score (R^2):", test_score)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Hyperparameters: {'activation': 'relu', 'hidden_layer_sizes': (512, 512, 512), 'learning_rate': 'adaptive', 'solver': 'adam'}
Test Score (R^2): 0.1923752586047588


In [27]:
# Evaluate the best model on the test set
test_predictions = model.predict(X_test)

# Calculate additional metrics
test_mse = mean_squared_error(y_test, test_predictions)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, test_predictions)
test_r2 = r2_score(y_test, test_predictions)

# Print the results
print(f"Test MSE: {test_mse}")
print(f"Test RMSE: {test_rmse}")
print(f"Test MAE: {test_mae}")
print(f"Test R^2: {test_r2}")


Test MSE: 0.3812762971349799
Test RMSE: 0.6174757461916864
Test MAE: 0.4129440122100839
Test R^2: 0.1923752586047588


### Without grid search

In [15]:
#### CREATING MODEL ####

# Define the neural network architecture with multiple hidden layers
class SolubilityPredictor(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super(SolubilityPredictor, self).__init__()
        layers = []
        previous_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(previous_dim, hidden_dim))
            layers.append(nn.ReLU())
            previous_dim = hidden_dim
        layers.append(nn.Linear(previous_dim, output_dim))
        self.network = nn.Sequential(*layers)
        self.activation = nn.ReLU() #best according to grid search

    def forward(self, x):
        return self.network(x)

# Define hyperparameters
input_dim = X_data.shape[1]  # Number of molecular descriptors
hidden_dim = [512,512,512]  # (512,512,512) Best according to grid search
output_dim = 1
learning_rate = 0.001
num_epochs = 1000
batch_size = 32

# Create DataLoader for batch training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize model, loss function (nn.MSELoss) and optimizer
model = SolubilityPredictor(input_dim, hidden_dim, output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss}")

# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
test_score = r2_score(y_test, y_pred)
print("Test Score (R^2):", test_score)

#maybe make plots of training and validation loss to see how it is learning
#can help identify overfitting

Epoch 1/1000, Loss: 729.7336807250977
Epoch 2/1000, Loss: 346.64645048975945
Epoch 3/1000, Loss: 291.02855122089386
Epoch 4/1000, Loss: 267.08760699629784
Epoch 5/1000, Loss: 246.62795370817184
Epoch 6/1000, Loss: 235.8152639567852
Epoch 7/1000, Loss: 225.01872636377811
Epoch 8/1000, Loss: 209.39354148507118
Epoch 9/1000, Loss: 196.32585054636002
Epoch 10/1000, Loss: 192.96081165969372
Epoch 11/1000, Loss: 175.84368343651295
Epoch 12/1000, Loss: 175.19147880375385
Epoch 13/1000, Loss: 171.55826039612293
Epoch 14/1000, Loss: 170.69676826894283
Epoch 15/1000, Loss: 158.91507704555988
Epoch 16/1000, Loss: 151.8885094821453
Epoch 17/1000, Loss: 139.26420390605927
Epoch 18/1000, Loss: 137.86115016043186
Epoch 19/1000, Loss: 141.31398610770702
Epoch 20/1000, Loss: 130.93004658818245
Epoch 21/1000, Loss: 130.66855208575726
Epoch 22/1000, Loss: 126.71719074994326
Epoch 23/1000, Loss: 125.44122186303139
Epoch 24/1000, Loss: 113.85645335912704
Epoch 25/1000, Loss: 111.33357813954353
Epoch 26/100

In [16]:
model.eval() # Set the model to evaluation mode

# Perform the forward pass to get predictions
with torch.no_grad():
    test_predictions_tensor = model(X_test_tensor)

# Convert predictions back to NumPy array
test_predictions = test_predictions_tensor.numpy().flatten()

# Calculate additional metrics
test_mse = mean_squared_error(y_test, test_predictions)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, test_predictions)
test_r2 = r2_score(y_test, test_predictions)

# Print the results
print(f"Test MSE: {test_mse}")
print(f"Test RMSE: {test_rmse}")
print(f"Test MAE: {test_mae}")
print(f"Test R^2: {test_r2}")

Test MSE: 1.2378835678100586
Test RMSE: 1.1126021146774292
Test MAE: 0.7025749683380127
Test R^2: 0.7718037850934754


### With dropout, weight decay
try dropout rates between 0.2 and 0.5. start with 0.5

In [54]:
# Define the neural network architecture with multiple hidden layers and dropout
class SolubilityPredictor(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super(SolubilityPredictor, self).__init__()
        layers = []
        previous_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(previous_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.2))  # Adding dropout
            previous_dim = hidden_dim
        layers.append(nn.Linear(previous_dim, output_dim))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# Define hyperparameters
input_dim = X_data.shape[1]  # Number of molecular descriptors
hidden_dims = [512, 512, 512]  # Best according to grid search
output_dim = 1
learning_rate = 0.001
num_epochs = 1000
batch_size = 32

# Create DataLoader for batch training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize model, loss function (nn.MSELoss) and optimizer with weight decay (L2 regularization)
model = SolubilityPredictor(input_dim, hidden_dims, output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    if epoch % 100 == 0:  # Print loss every 100 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss}")

# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor).numpy().flatten()

test_mse = mean_squared_error(y_test, y_pred)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, y_pred)
test_r2 = r2_score(y_test, y_pred)

print(f"Test MSE: {test_mse}")
print(f"Test RMSE: {test_rmse}")
print(f"Test MAE: {test_mae}")
print(f"Test R^2: {test_r2}")


Epoch 1/1000, Loss: 681.7832328081131
Epoch 101/1000, Loss: 111.76737007498741
Epoch 201/1000, Loss: 74.74509523808956
Epoch 301/1000, Loss: 64.90365134179592
Epoch 401/1000, Loss: 56.54144813865423
Epoch 501/1000, Loss: 55.897232234478
Epoch 601/1000, Loss: 51.54749147221446
Epoch 701/1000, Loss: 52.32152093201876
Epoch 801/1000, Loss: 46.192381370812654
Epoch 901/1000, Loss: 48.13287205994129
Test MSE: 1.2193962335586548
Test RMSE: 1.1042627096176147
Test MAE: 0.695110559463501
Test R^2: 0.7752118032844304


### Saving the best model

In [17]:
torch.save(model.state_dict(), 'solubility_model.pth')
